# Results

This notebook gathers results from evaluation JSON files and prints them as a list. 

### Setup

- Set the root folder environment variable with `export CLIPORT_ROOT=<cliport_root>`
- Train and evaluate agents by following the [README guide](https://github.com/cliport/cliport#single-task-training--evaluation).

In [1]:
import os
import sys
import json

from cliport import agents
from cliport import tasks

pybullet build time: Aug 16 2021 17:58:31


### Settings

In [2]:
root_folder = os.environ['CLIPORT_ROOT']
exp_folder = os.path.join(root_folder, 'cliport_quickstart') # replace 'cliport_quickstart' with your exps folder

### Gather JSON Results

In [3]:
tasks_list = list(tasks.names.keys())
agents_list = list(agents.names.keys())
demos_list = [1, 10, 100, 1000]

results = {}
for t in tasks_list:
    for a in agents_list:
        for d in demos_list:
            task_folder = f'{t}-{a}-n{d}-train'
            task_folder_path = os.path.join(exp_folder, task_folder, 'checkpoints')

            if os.path.exists(task_folder_path):
                jsons = [f for f in os.listdir(task_folder_path) if '.json' in f]
                for j in jsons:
                    model_type = 'multi' if 'multi' in j else 'single'
                    eval_type = 'val' if 'val' in j else 'test'
                    
                    with open(os.path.join(task_folder_path, j)) as f:
                        res = json.load(f)
                    
                    results[f'{t}-{a}-n{d}-{model_type}-{eval_type}'] = res

### Print Results

In [6]:
print(f'Experiments folder: {exp_folder}\n')

for eval_type in ['val', 'test']:
    print(f'----- {eval_type.upper()} -----\n')
    for t in tasks_list:
        for a in agents_list:
            for d in demos_list:
                for model_type in ['single', 'multi']:
                    eval_key = f'{t}-{a}-n{d}-{model_type}-{eval_type}'
                    
                    if eval_key in results:    
                        print(f'{t} | Train Demos: {d}')
                        
                        res = results[eval_key]
                        best_score, best_ckpt = max(zip([v['mean_reward'] for v in list(res.values())], 
                                                        res.keys())) # TODO: test that this works for full results folder
                        
                        print(f'\t{best_score*100:1.1f} : {a} | {model_type}\n')
                            

Experiments folder: /home/mshr/cliport/cliport_quickstart

----- VAL -----

stack-block-pyramid-seq-seen-colors | Train Demos: 1000
	97.3 : cliport | multi

----- TEST -----

stack-block-pyramid-seq-seen-colors | Train Demos: 1000
	96.5 : cliport | multi

